In [1]:
import os
os.environ['ONANDA_ENV'] = 'practice'
os.environ['OANDA_ACCOUNT_ID']=''
os.environ['OANDA_ENV']=''
os.environ['OANDA_ACCESS_TOKEN']=''
os.environ['MONGODB_HOSTS'] = ''
os.environ['MONGODB_USER'] = ''
os.environ['MONGODB_PASSWORD'] = ''

In [2]:
from trading.algorithms.moving_average_crossover import MAC
from trading.broker.oanda import OandaBroker
from trading.classifier.random_forest import RFClassifier
from trading.scripts.training import get_training_data
from trading.training_strategies.base import TrainingStrategy

In [3]:
instrument = 'EUR_USD'
count = 20 # number of data points to look back
granularity = 'D' # Day
pair_a = {'name': 'EUR', 'amount': 1000}
pair_b = {'name': 'USD', 'amount': 1000}
config = {
    'instrument': instrument,
    'pair_a': pair_a,
    'pair_b': pair_b
}
strategy_id = None

In [6]:
strategy = MAC(config, strategy_id)
broker = OandaBroker()
training_strategy = TrainingStrategy(strategy, broker, instrument, granularity, count)

[INFO] message: Starting Portfolio data: Instrument EUR_USD Profit 0 Pair A: {'amount': 1000, 'initial_currency': 1000, 'name': 'EUR', 'tradeable_currency': 1000} Pair B: {'amount': 1000, 'initial_currency': 1000, 'name': 'USD', 'tradeable_currency': 1000}


In [7]:
strategy_data = training_strategy.tick()

[INFO] message: Tick Number: 0
[INFO] message: Order decision buy and market order MarketOrder(instrument='EUR_USD', units=895.0708448573705, side='buy', type='market', price=1.11723, expiry='2016-04-24T22:00:30.115957Z')
[INFO] message: Order Response
[INFO] message: Tick Number: 1
[INFO] message: Order decision buy and market order MarketOrder(instrument='EUR_USD', units=895.0708448573705, side='buy', type='market', price=1.11723, expiry='2016-04-24T22:00:30.116743Z')
[INFO] message: Order Response
[INFO] message: Tick Number: 2
[INFO] message: Order decision buy and market order MarketOrder(instrument='EUR_USD', units=895.0708448573705, side='buy', type='market', price=1.11723, expiry='2016-04-24T22:00:30.117275Z')
[INFO] message: Order Response
[INFO] message: Tick Number: 3
[INFO] message: Order decision buy and market order MarketOrder(instrument='EUR_USD', units=895.0708448573705, side='buy', type='market', price=1.11723, expiry='2016-04-24T22:00:30.117778Z')
[INFO] message: Ord

In [8]:
from collections import OrderedDict

In [9]:
strategy_data = OrderedDict(strategy_data)

In [29]:
strategy_data

OrderedDict([(0,
              {'decision': 'buy',
               'long_term_ma': Decimal('1.1098285000000005506848310687928460538387298583984375'),
               'short_term_ma': Decimal('1.120240999999999598202293782378546893596649169921875')}),
             (1,
              {'decision': 'buy',
               'long_term_ma': Decimal('1.109828500000000328640226143761537969112396240234375'),
               'short_term_ma': Decimal('1.120240999999999598202293782378546893596649169921875')}),
             (2,
              {'decision': 'buy',
               'long_term_ma': Decimal('1.109828500000000328640226143761537969112396240234375'),
               'short_term_ma': Decimal('1.12024100000000004229150363244116306304931640625')}),
             (3,
              {'decision': 'buy',
               'long_term_ma': Decimal('1.109828500000000328640226143761537969112396240234375'),
               'short_term_ma': Decimal('1.120240999999999598202293782378546893596649169921875')}),
           

In [11]:
config = {'classifier_id': None}
rfc = RFClassifier(config)

In [12]:
X, y = rfc.prepare_training_data(strategy_data)

[INFO] message: Training Features data: ['long_term_ma', 'decision', 'short_term_ma']


In [14]:
rfc.train(X,y)

In [17]:
s = rfc.serialize()

In [19]:
from bson import ObjectId

In [22]:
from trading.db import get_database

In [26]:
db = get_database()

In [28]:
db.classifiers.insert({'name': 'rfc', 'classifier': s})

/Users/jnewman/Projects/Trading_Env/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


ObjectId('571bf11f16890198e1e0243d')